--> **Make sure to have configure aws cli** <--

Sources:
* https://medium.com/rapids-ai/getting-started-with-rapids-on-aws-ecs-using-dask-cloud-provider-b1adfdbc9c6e
* https://cloudprovider.dask.org/en/latest/aws.html

In [ ]:
%pip install dask-cloudprovider

In [2]:
from dask_cloudprovider.aws import ECSCluster

In [ ]:
# change ***** for aws user id
cluster = ECSCluster(
    cluster_arn="arn:aws:ecs:us-east-1:*****:cluster/dask-cluster",
    n_workers=1,
    fargate_scheduler=True    
)

This error (edited) was produced:

`RuntimeError: {'tasks': [], 'failures': [{'arn': 'arn:aws:ecs:us-east-1:***:container-instance/e1e87bfb484447e4a5557f54201675af', 'reason': 'RESOURCE:CPU'}], 'ResponseMetadata': {'RequestId': 'request_id****', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'request_id****', 'content-type': 'application/x-amz-json-1.1', 'content-length': '146', 'date': 'Mon, 21 Mar 2022 10:37:10 GMT'}, 'RetryAttempts': 0}}`

In [2]:
from dask_cloudprovider.aws import FargateCluster

In [ ]:
# ERROR: InvalidParameterException: An error occurred (InvalidParameterException) when calling the RunTask operation: The specified capacity provider strategy cannot contain a capacity provider that is not associated with the cluster. Associate the capacity provider with the cluster or specify a valid capacity provider and try again.
# cluster = FargateCluster(n_workers=1, fargate_spot=True) 
# cluster = FargateCluster(fargate_spot=True)
cluster = FargateCluster(n_workers=1)

In [14]:
cluster

In [15]:
cluster.scale(2)

In [16]:
from dask.distributed import Client
client = Client(cluster)

/Users/eduardogutierrez/opt/anaconda3/envs/dask-recipes/lib/python3.8/site-packages/distributed/client.py:1105: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| blosc       | None      | 1.10.2    | 1.10.2    |
| dask        | 2021.08.0 | 2022.02.1 | 2022.02.1 |
| distributed | 2021.08.0 | 2022.2.1  | 2022.2.1  |
| lz4         | None      | 3.1.10    | 3.1.10    |
| numpy       | 1.22.3    | 1.21.1    | 1.21.1    |
| pandas      | 1.4.1     | 1.3.0     | 1.3.0     |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


**There is a version mismatch!**. Follow guidelines here:
https://cloudprovider.dask.org/en/latest/aws.html#dask_cloudprovider.aws.FargateCluster

One strategy to ensure that package versions match between your custom environment and the Docker container is to create your environment from an `environment.yml` file, export the exact package list for that environment using `conda list --export > package-list.txt`, and then use the pinned package versions contained in `package-list.txt` in your Dockerfile. You could use the default [Dask Dockerfile](https://github.com/dask/dask-docker/blob/main/base/Dockerfile) as a template and simply add your pinned additional packages.

In [17]:
# quick test
import dask.array as da
arr = da.random.random((1000, 1000, 10000), chunks=(100, 100, 1000))
arr = arr.mean().persist()

In [19]:
arr

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [20]:
cluster.scale(1) # check Dask dashboard

In [21]:
arr = da.random.random((1000, 1000, 10000), chunks=(100, 100, 1000))
arr = arr.mean().persist()

In [22]:
arr

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

Cluster is deleted by default after 5 minutes of inactivity.

Still, it is necessary to ensure they correctly shut down! Check (depending on aws cli credentials region) https://us-east-1.console.aws.amazon.com/ecs/